In [1]:
from crawler import craw
craw.fetch_departments()

[Work] Fetching departments ......
Parsing Chinese department data ......
Parsing Chinese department data ......
[Done] All departments fetched ......


([{'name': {'en': 'College of Liberal Arts', 'ch': '文學院'}, 'serial': 0},
  {'name': {'en': 'College of Science', 'ch': '理學院'}, 'serial': 1},
  {'name': {'en': 'College of Engineering', 'ch': '工學院'}, 'serial': 2},
  {'name': {'en': 'College of Management', 'ch': '管理學院'}, 'serial': 3},
  {'name': {'en': 'College of Electrical Engineering and Computer Science',
    'ch': '資電學院'},
   'serial': 4},
  {'name': {'en': 'College of Earth Sciences', 'ch': '地科學院'}, 'serial': 5},
  {'name': {'en': 'College of Hakka Studies', 'ch': '客家學院'}, 'serial': 6},
  {'name': {'en': 'College of Health Sciences and Technology', 'ch': '生醫理工學院'},
   'serial': 7},
  {'name': {'en': 'College of Graduate College of Sustainability and Green Energy',
    'ch': '永續與綠能科技研究學院'},
   'serial': 8}],
 [{'name': {'en': 'Center of Teacher-training Program', 'ch': '師資培育中心'},
   'facility': 0,
   'url': 'https://cis.ncu.edu.tw/Course/main/query/byUnion?dept=deptI0I6I0',
   'course_cnt': 17},
  {'name': {'en': 'Department of Chi